### Copyright (C) Infineon Technologies AG 2025
 
Copyright (c) 2025, Infineon Technologies AG, or an affiliate of Infineon Technologies AG. All rights reserved.
This software, associated documentation and materials ("Software") is owned by Infineon Technologies AG or one of its affiliates ("Infineon") and is protected by and subject to worldwide patent protection, worldwide copyright laws, and international treaty provisions. Therefore, you may use this Software only as provided in the license agreement accompanying the software package from which you obtained this Software. If no license agreement applies, then any use, reproduction, modification, translation, or compilation of this Software is prohibited without the express written permission of Infineon.

Disclaimer: UNLESS OTHERWISE EXPRESSLY AGREED WITH INFINEON, THIS SOFTWARE IS PROVIDED AS-IS, WITH NO WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING, BUT NOT LIMITED TO, ALL WARRANTIES OF NON-INFRINGEMENT OF THIRD-PARTY RIGHTS AND IMPLIED WARRANTIES SUCH AS WARRANTIES OF FITNESS FOR A SPECIFIC USE/PURPOSE OR MERCHANTABILITY. Infineon reserves the right to make changes to the Software without notice. You are responsible for properly designing, programming, and testing the functionality and safety of your intended application of the Software, as well as complying with any legal requirements related to its use. Infineon does not guarantee that the Software will be free from intrusion, data theft or loss, or other breaches ("Security Breaches"), and Infineon shall have no liability arising out of any Security Breaches. Unless otherwise explicitly approved by Infineon, the Software may not be used in any application where a failure of the Product or any consequences of the use thereof can reasonably be expected to result in personal injury.

# Remaining Useful Life (RUL) Prediction with LSTM

This notebook demonstrates how to build and train a Long Short-Term Memory (LSTM) neural network for remaining useful life prediction using the NASA Turbofan Engine dataset and the helper functions from `modelling_helper.py`.

## Introduction

Remaining Useful Life (RUL) prediction is a key technology for predictive maintenance systems across industrial sectors. In automotive applications, accurate RUL estimation provides:

- **Proactive maintenance scheduling** - Reducing unplanned downtime through failure prediction
- **Cost optimization** - Extending component operational life while avoiding premature replacement
- **Safety improvement** - Early identification of potential component failures
- **Fleet optimization** - Data-driven maintenance scheduling across vehicle fleets
- **Warranty analysis** - Understanding component degradation patterns for design optimization

LSTM networks are particularly well-suited for RUL prediction as they can capture long-term dependencies in time series data and model complex degradation patterns.

### Notebook Structure

1. **Import Libraries and Helper Functions**
2. **Load and Explore the NASA Turbofan Engine Dataset**
3. **Data Visualization**
4. **Data Preprocessing for LSTM**
5. **Create and Build LSTM Model**
6. **Train the Model**
7. **Model Export and Conversion**
8. **Model Evaluation**
9. **Compiling the Model for AURIX&trade;**
10. **Evaluate Execution Timing**

## Import Libraries and Helper Functions

In [ ]:
import sys
import os

# Add the parent directory to the Python path so we can import from central_scripts
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from CentralScripts.helper_functions import *
from modelling_helper import *

origin = "torch"

train_data_path = "data/train_FD001.txt"
test_data_path = "data/test_FD001.txt"
test_RUL_path = "data/RUL_FD001.txt"

## Load and Investigate the NASA Turbofan Engine Dataset

This project uses the **NASA Turbofan Engine Degradation Simulation Dataset** (also known as the NASA C-MAPSS dataset). The dataset contains:

- **Training data**: Run-to-failure time series data from multiple aircraft engines
- **Test data**: Partial time series data for engines with unknown remaining useful life
- **Ground truth**: Actual RUL values for the test engines

The data loading process includes:
- **Automatic download**: If data files are not found locally, they are automatically downloaded from the official NASA repository
- **Data normalization**: For better model training the train data is normalized
- **Feature filtering**: Remove features that do not carry meaningful information by checking the standard deviation of the data. Features with a standard deviation lower than 0.02 are excluded

**Data Source**: The dataset is automatically downloaded from the [NASA Prognostics Data Repository](https://www.nasa.gov/content/prognostics-center-of-excellence-data-set-repository) when needed.

 A. Saxena and K. Goebel (2008). “Turbofan Engine Degradation Simulation Data Set”, NASA Prognostics Data Repository, NASA Ames Research Center, Moffett Field, CA

In [ ]:
# Load and process the training data
# Note: If data files are not found locally, they will be automatically downloaded from NASA
train_data = load_dataframe(train_data_path)
train_data, scaler = normalize_data(train_data)

removed_cols = clean_data(train_data)
train_data.drop(removed_cols, axis=1, inplace=True)
train_data = add_rul(train_data)

print(f"Training data shape: {train_data.shape}")
print(f"Removed columns: {removed_cols}")
train_data.head()

## Data Visualization

- Time series patterns
- RUL distribution
- Sensor correlation analysis

In [ ]:
sensor_cols = [
    col for col in train_data.columns if "sensor" in col or "op_setting" in col
]

fig, axes = plt.subplots(len(sensor_cols), 1, figsize=(12, 15), sharex=True)

for i, sensor in enumerate(sensor_cols):
    for unit in train_data["unit_number"].unique():
        unit_data = train_data[train_data["unit_number"] == unit]
        axes[i].plot(unit_data["time_in_cycles"], unit_data[sensor])
    axes[i].set_ylabel(sensor)

axes[-1].set_xlabel("Time in Cycles")
plt.tight_layout()
plt.show()

## Data Preprocessing for LSTM

- Turn into sequences of a defined length. The training data will have the shape `n_samples * sequence_length * n_features`
- Split into training and validation data

In [ ]:
from sklearn.model_selection import train_test_split

sequence_length = 50
# Prepare sequences
X, y = prepare_train_sequences(train_data, sequence_length)

# Split the data into 90% training and 10% validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"Sequence length: {X_train.shape[1]}")
print(f"Number of features: {X_train.shape[2]}")

## Create and Build LSTM Model

- set `num_layers` and `hidden_size` to tweak the architecture and test its effects on predictions

In [ ]:
from torchinfo import summary

# LSTM architecture parameters
input_size = X_train.shape[2]  # Number of features
hidden_size = 16  # Hidden units in LSTM
num_layers = 2  # Number of LSTM layers
output_size = 1  # Single RUL value
dropout = 0.05  # Dropout rate

model = LSTMmodel(
    input_size=input_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    output_size=output_size,
    dropout_rate=dropout,
)

# Print model summary
print(f"Model type: {model.model_type}")
print(f"Input size: {input_size}")
print(f"Hidden size: {hidden_size}")
print(f"Number of layers: {num_layers}")
summary(model)

# Train the model

- The training process uses early stopping to avoid overfitting.
- To ensure proper convergence learning rate reduction is used when training stalls.

In [ ]:
# Use original input data for LSTM (no flattening)
X_train_final = X_train
X_val_final = X_val

print("Using original input for LSTM:")
print(f"X_train_final shape: {X_train_final.shape}")
print(f"X_val_final shape: {X_val_final.shape}")

train_loss, val_loss, lr = train_model(
    model=model,
    X_train=X_train_final,
    y_train=y_train,
    X_val=X_val_final,
    y_val=y_val,
    initial_lr=0.01,
    num_epochs=10,  # Reduced for faster training in demo
)

plot_training_progress(train_loss, val_loss, lr)

# Load the best model - check if file exists first
checkpoint_path = "model_checkpoints/best_model.pth"
if os.path.exists(checkpoint_path):
    model.load_state_dict(torch.load(checkpoint_path))
    print("Best model loaded.")
else:
    print("No checkpoint found. Using current model state.")

# Export Model to ONNX

- Generate a model name based on its architecture and loss
- Save the model together with a test in- & output

In [ ]:
# Generate model name
model_name = f"{model.model_type}_layers-{num_layers}_loss-{min(val_loss):.1f}"
model_name = model_name.replace(".", "-")

# Prepare input and output for model export
input_target = X_train[0].astype(np.float32)  # Single sequence (no flattening for LSTM)
output_target = get_predictions(origin, model, input_target)

save_all(model_name, input_target, output_target, model, origin)

print(f"Model saved as: {model_name}")
print(f"Input shape for export: {input_target.shape}")
print(f"Output shape: {output_target.shape}")

## Model Evaluation

### Load Test Data

- Data that was not used during model training.
- Normalize it with the scaler from the train data
- Generate sequences of the same length as for training

In [ ]:
test_data = load_dataframe(test_data_path)
test_RUL = pd.read_csv(test_RUL_path, names=["RUL"], header=None)
test_data, scaler = normalize_data(test_data, scaler)

test_data.drop(removed_cols, axis=1, inplace=True)

X_test, y_test = prepare_test_sequences(test_data, test_RUL, sequence_length)

print(f"Test data shape: {test_data.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

test_data

### Make Model Predictions

In [ ]:
model_folder, onnx_model_file = get_output_paths(model_name)
y_pred_test = pd.DataFrame(
    {
        "y_pred": predict_with_onnx(onnx_model_file, X_test).reshape(
            -1,
        ),
        "y": y_test,
        "Train_test": "Test",
    }
)

y_pred_train = pd.DataFrame(
    {
        "y_pred": predict_with_onnx(onnx_model_file, X_train).reshape(
            -1,
        ),
        "y": y_train,
        "Train_test": "Train",
    }
)

y_pred = pd.concat([y_pred_train, y_pred_test])

### Performance Analysis

- Mean squared error (MSE) as loss
- R2 as statistical metric for the predictive quality of the model, R2 = 1 indicating a perfect model
- Compare the performance on train and test data

In [ ]:
kpi_df = calculate_kpis(y_pred)
print("Model Performance Metrics:")
print(kpi_df)

### Plot true vs. predicted RUL

- Assess model performance by directly comparing true and predicted values
- Better correlation indicates a better model

In [ ]:
plot_true_vs_predicted(y_pred, kpi_df)

# Convert Model for AURIX&trade; TC3x and TC4x deployment

- Make sure the docker container with conversion tools is running
- Submit the model together with test data to the container and download the generated code, binary and log files.
- Results are saved in the `out/<model_name>/test_<model_name>/<target>/` folder

In [ ]:
ensure_docker_container()

In [ ]:
from CentralScripts.python_flask_client import CallTools

model_folder, onnx_model_file = get_output_paths(model_name)

for target in ["TC3", "TC4"]:
    tool = CallTools(
        folder=model_folder, url="http://localhost:8080/convert", target=target
    )
    tool.convert_model()

## Simulated instruction counts

- For each node in the neural network the number of instruction counts is extracted from the log file and plotted.
- You can inspect which node is a computational bottleneck and adjust your network.

In [ ]:
plot_instruction_counts(model_name)